# 코로나 월별 감염자, 예방접종자 수 합 


- (코로나 발생 월)부터 (현재 월 - 1) 월별로 합을 구한다

- 년, 월로 나눈 후 groupby 년도, 월.sum()

- 국내 코로나 확진자-백신접종자 현황_2021년 9월 3주차.xlsx

- 수정 : 감염현황 api에서 5/31이 추가되어 해당 부분 수정

In [2]:
s = '국내 코로나 확진자-백신접종자 지역별 일일 현황_2022년 2월 4주차.xlsx'

In [3]:
import pandas as pd
import openpyxl
import xlsxwriter

df = pd.read_excel(s, engine = 'openpyxl')
df

,시도명,날짜,당일 통계(1차 접종),전체 누적 통계(1차 접종),전일까지의 누적 통계 (1차 접종),당일 통계(2차 접종),전체 누적 통계(2차 접종),전일까지의 누적 통계 (2차 접종)
0,강원도,2021-03-11,2627,14790,12163,0,0,0
1,강원도,2021-03-12,2314,17144,14830,0,0,0
2,강원도,2021-03-13,1336,18481,17145,0,0,0
3,강원도,2021-03-14,72,18553,18481,0,0,0
4,강원도,2021-03-15,0,18553,18553,0,0,0
...,...,...,...,...,...,...,...,...
5979,충청북도,2022-02-21,4,1418181,1418177,18,1403400,1403382
5980,충청북도,2022-02-22,253,1418472,1418219,176,1403615,1403439
5981,충청북도,2022-02-23,241,1418757,1418516,130,1403796,1403666
5982,충청북도,2022-02-24,180,1418969,1418789,104,1403932,1403828


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5984 entries, 0 to 5983
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   시도명                  5984 non-null   object
 1   날짜                   5984 non-null   object
 2   당일 통계(1차 접종)         5984 non-null   int64 
 3   전체 누적 통계(1차 접종)      5984 non-null   int64 
 4   전일까지의 누적 통계 (1차 접종)  5984 non-null   int64 
 5   당일 통계(2차 접종)         5984 non-null   int64 
 6   전체 누적 통계(2차 접종)      5984 non-null   int64 
 7   전일까지의 누적 통계 (2차 접종)  5984 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 374.1+ KB


In [5]:
# float -> int
df = df.astype({'날짜': str})
# na 값을 0으로 대체
df = df.fillna(0)

In [6]:
# 당일 누적 1차 백신 접종자 수를 당일 일일 1차 백신 접종자 수로 변경
df['daily_FirstCnt'] = df['전체 누적 통계(1차 접종)'].diff()

# 당일 누적 2차 백신 접종자당일 일일 2차 백신 접종자 수로 변경
df['daily_SecondCnt'] = df['전체 누적 통계(2차 접종)'].diff()

# 날짜 월까지만 남기기
# 특정문자 기준으로 문자열 나누고 새로운 열 추가
df['년'] = df.날짜.str.split('-').str[0]
df['월'] = df.날짜.str.split('-').str[1]

# 날짜를 지우고 년 월만 맨 앞으로
new_df = df[['년', '월', '시도명','당일 통계(1차 접종)','전체 누적 통계(1차 접종)','전일까지의 누적 통계 (1차 접종)', '당일 통계(2차 접종)', '전체 누적 통계(2차 접종)','전일까지의 누적 통계 (2차 접종)']]

In [7]:
new_df

,년,월,시도명,당일 통계(1차 접종),전체 누적 통계(1차 접종),전일까지의 누적 통계 (1차 접종),당일 통계(2차 접종),전체 누적 통계(2차 접종),전일까지의 누적 통계 (2차 접종)
0,2021,03,강원도,2627,14790,12163,0,0,0
1,2021,03,강원도,2314,17144,14830,0,0,0
2,2021,03,강원도,1336,18481,17145,0,0,0
3,2021,03,강원도,72,18553,18481,0,0,0
4,2021,03,강원도,0,18553,18553,0,0,0
...,...,...,...,...,...,...,...,...,...
5979,2022,02,충청북도,4,1418181,1418177,18,1403400,1403382
5980,2022,02,충청북도,253,1418472,1418219,176,1403615,1403439
5981,2022,02,충청북도,241,1418757,1418516,130,1403796,1403666
5982,2022,02,충청북도,180,1418969,1418789,104,1403932,1403828


In [8]:
result_df = new_df.groupby(['년', '월', '시도명'], as_index=False).sum()
result_df

,년,월,시도명,당일 통계(1차 접종),전체 누적 통계(1차 접종),전일까지의 누적 통계 (1차 접종),당일 통계(2차 접종),전체 누적 통계(2차 접종),전일까지의 누적 통계 (2차 접종)
0,2021,03,강원도,12602,441262,428660,817,1326,509
1,2021,03,경기도,77387,3039484,2962097,647,2361,1714
2,2021,03,경상남도,21282,1087710,1066428,524,2441,1917
3,2021,03,경상북도,18530,768308,749778,0,0,0
4,2021,03,광주광역시,15629,603335,587706,191,610,419
...,...,...,...,...,...,...,...,...,...
199,2022,02,전라남도,4779,40981449,40976670,9968,40537823,40527855
200,2022,02,전라북도,4601,39510582,39505981,8761,39118732,39109971
201,2022,02,제주특별자치도,2691,14597596,14594905,4526,14415016,14410490
202,2022,02,충청남도,6550,47015462,47008912,13804,46457499,46443695


In [9]:
population = 51780579
result_df['누적 1차 백신 접종률'] = result_df['전체 누적 통계(1차 접종)'].apply(lambda e: (e/population))
result_df['누적 2차 백신 접종률'] = result_df['전체 누적 통계(2차 접종)'].apply(lambda e: (e/population))

In [1]:
204595290/51780579

3.9511974170856607

In [10]:
result_df

,년,월,시도명,당일 통계(1차 접종),전체 누적 통계(1차 접종),전일까지의 누적 통계 (1차 접종),당일 통계(2차 접종),전체 누적 통계(2차 접종),전일까지의 누적 통계 (2차 접종),누적 1차 백신 접종률,누적 2차 백신 접종률
0,2021,03,강원도,12602,441262,428660,817,1326,509,0.008522,0.000026
1,2021,03,경기도,77387,3039484,2962097,647,2361,1714,0.058699,0.000046
2,2021,03,경상남도,21282,1087710,1066428,524,2441,1917,0.021006,0.000047
3,2021,03,경상북도,18530,768308,749778,0,0,0,0.014838,0.000000
4,2021,03,광주광역시,15629,603335,587706,191,610,419,0.011652,0.000012
...,...,...,...,...,...,...,...,...,...,...,...
199,2022,02,전라남도,4779,40981449,40976670,9968,40537823,40527855,0.791444,0.782877
200,2022,02,전라북도,4601,39510582,39505981,8761,39118732,39109971,0.763039,0.755471
201,2022,02,제주특별자치도,2691,14597596,14594905,4526,14415016,14410490,0.281913,0.278387
202,2022,02,충청남도,6550,47015462,47008912,13804,46457499,46443695,0.907975,0.897199


In [11]:
result_df[result_df['시도명'] == '강원도']

,년,월,시도명,당일 통계(1차 접종),전체 누적 통계(1차 접종),전일까지의 누적 통계 (1차 접종),당일 통계(2차 접종),전체 누적 통계(2차 접종),전일까지의 누적 통계 (2차 접종),누적 1차 백신 접종률,누적 2차 백신 접종률
0,2021,03,강원도,12602,441262,428660,817,1326,509,0.008522,0.000026
17,2021,04,강원도,87973,1721364,1633391,11395,82847,71452,0.033243,0.001600
34,2021,05,강원도,80784,4834565,4753781,66550,1393849,1327299,0.093366,0.026918
51,2021,06,강원도,315403,13053473,12738070,91297,3621379,3530082,0.252092,0.069937
68,2021,07,강원도,109463,17993689,17884226,61328,7136068,7074740,0.347499,0.137814
85,2021,08,강원도,274574,24414394,24139820,254166,11438263,11184097,0.471497,0.220899
102,2021,09,강원도,246375,32017321,31770946,287416,20890245,20602829,0.618327,0.403438
119,2021,10,강원도,59224,37514226,37455002,345052,31233831,30888779,0.724484,0.603196
136,2021,11,강원도,40667,37764953,37724286,66065,36078051,36011986,0.729327,0.696749
153,2021,12,강원도,43833,40257621,40213788,42512,38964155,38921643,0.777466,0.752486


In [12]:
# df_conf = df_conf[''].groupby(df_conf[''])
grouped = result_df.groupby(['시도명', '년', '월'], as_index=False).sum()
grouped

,시도명,년,월,당일 통계(1차 접종),전체 누적 통계(1차 접종),전일까지의 누적 통계 (1차 접종),당일 통계(2차 접종),전체 누적 통계(2차 접종),전일까지의 누적 통계 (2차 접종),누적 1차 백신 접종률,누적 2차 백신 접종률
0,강원도,2021,03,12602,441262,428660,817,1326,509,0.008522,0.000026
1,강원도,2021,04,87973,1721364,1633391,11395,82847,71452,0.033243,0.001600
2,강원도,2021,05,80784,4834565,4753781,66550,1393849,1327299,0.093366,0.026918
3,강원도,2021,06,315403,13053473,12738070,91297,3621379,3530082,0.252092,0.069937
4,강원도,2021,07,109463,17993689,17884226,61328,7136068,7074740,0.347499,0.137814
...,...,...,...,...,...,...,...,...,...,...,...
199,충청북도,2021,10,68353,39589867,39521514,411139,31899423,31488284,0.764570,0.616050
200,충청북도,2021,11,45051,39906677,39861626,75846,37999809,37923963,0.770688,0.733862
201,충청북도,2021,12,47171,42530310,42483139,44744,41147524,41102780,0.821356,0.794652
202,충청북도,2022,01,14623,43628347,43613724,46067,42579552,42533485,0.842562,0.822307


In [13]:
# 유일한 값 찾기
city_list = grouped['시도명'].unique().tolist()
city_list

['강원도',
 '경기도',
 '경상남도',
 '경상북도',
 '광주광역시',
 '대구광역시',
 '대전광역시',
 '부산광역시',
 '서울특별시',
 '세종특별자치시',
 '울산광역시',
 '인천광역시',
 '전라남도',
 '전라북도',
 '제주특별자치도',
 '충청남도',
 '충청북도']

In [14]:
df_list = []

for city in city_list:
    df_list.append(grouped[grouped['시도명']==city])

In [15]:
import datetime as dt
from datetime import datetime, timedelta

def get_date(y, m, d):
    s = f'{y:04d}-{m:02d}-{d:02d}'
    return datetime.strptime(s, '%Y-%m-%d')

def get_week_no(y, m, d):
    target = get_date(y, m, d)
    firstday = target.replace(day = 1)
    if firstday.weekday() == 6:
        origin = firstday
    elif firstday.weekday() < 3:
        origin = firstday - timedelta(days=firstday.weekday() + 1)
    else:
        origin = firstday + timedelta(days = 6-firstday.weekday())
    return (target - origin).days // 7 + 1

In [16]:
n = dt.datetime.now()

writer = pd.ExcelWriter('국내 코로나 확진자-백신접종자 지역별 월별 접종률_{}년 {}월 {}주차.xlsx'
                        .format(n.year, n.month, get_week_no(n.year, n.month, n.day)), 
                        engine = 'xlsxwriter')

In [17]:
for i, df in enumerate(df_list):
    print(city_list[i])
    print(df)
    df.to_excel(writer, index = False, sheet_name = city_list[i])
    
writer.save()

강원도
    시도명     년   월  당일 통계(1차 접종)  전체 누적 통계(1차 접종)  전일까지의 누적 통계 (1차 접종)  \
0   강원도  2021  03         12602           441262               428660   
1   강원도  2021  04         87973          1721364              1633391   
2   강원도  2021  05         80784          4834565              4753781   
3   강원도  2021  06        315403         13053473             12738070   
4   강원도  2021  07        109463         17993689             17884226   
5   강원도  2021  08        274574         24414394             24139820   
6   강원도  2021  09        246375         32017321             31770946   
7   강원도  2021  10         59224         37514226             37455002   
8   강원도  2021  11         40667         37764953             37724286   
9   강원도  2021  12         43833         40257621             40213788   
10  강원도  2022  01         13118         41249225             41236107   
11  강원도  2022  02          4363         33453771             33449408   

    당일 통계(2차 접종)  전체 누적 통계(2차 접종)  전일까지의 누적 통계